In [174]:
import pandas as pd

# קריאה של הקובץ
df_stat_fighter = pd.read_csv("/content/ufc_fight_stats.csv")
#df_main = pd.read_csv("/content/ufc_fight_details.csv")
df_result = pd.read_csv("/content/ufc_fight_results.csv")
df_tott = pd.read_csv("/content/ufc_fighter_tott.csv")
df_event = pd.read_csv("/content/ufc_event_details.csv")

# הצגת 5 השורות הראשונות


In [175]:
df_result

,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,URL
0,UFC 318: Holloway vs. Poirier 3,Max Holloway vs. Dustin Poirier,W/L,Lightweight Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Jason Herzog,Derek Cleary 47 - 48.Sal D'amato 46 - 49.Eric ...,http://ufcstats.com/fight-details/d7507e2741fd...
1,UFC 318: Holloway vs. Poirier 3,Paulo Costa vs. Roman Kopylov,W/L,Middleweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Herb Dean,Derek Cleary 27 - 30.Eric Colon 27 - 30.Chris ...,http://ufcstats.com/fight-details/23c963be9de3...
2,UFC 318: Holloway vs. Poirier 3,Kevin Holland vs. Daniel Rodriguez,L/W,Welterweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Mike Beltran,Sal D'amato 28 - 29.Junichiro Kamijo 28 - 29.C...,http://ufcstats.com/fight-details/5dde61734080...
3,UFC 318: Holloway vs. Poirier 3,Dan Ige vs. Patricio Freire,L/W,Featherweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Dan Miragliotta,Eric Colon 28 - 29.Sal D'amato 28 - 29.Junichi...,http://ufcstats.com/fight-details/acc961fd5cf5...
4,UFC 318: Holloway vs. Poirier 3,Michael Johnson vs. Daniel Zellhuber,W/L,Lightweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Kerry Hatley,Derek Cleary 28 - 29.Eric Colon 28 - 29.Junich...,http://ufcstats.com/fight-details/a95b03e9b5a9...
...,...,...,...,...,...,...,...,...,...,...,...
8209,UFC 2: No Way Out,Orlando Wiet vs. Robert Lucarelli,W/L,Open Weight Bout,KO/TKO,1,2:50,No Time Limit,John McCarthy,toCorner Stoppage,http://ufcstats.com/fight-details/3b020d4914b4...
8210,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,W/L,Open Weight Bout,Submission,1,4:52,No Time Limit,John McCarthy,Keylock From Half Guard,http://ufcstats.com/fight-details/d917c8c7461b...
8211,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,W/L,Open Weight Bout,KO/TKO,1,12:13,No Time Limit,John McCarthy,Punches to Head From GuardSubmission to Strikes,http://ufcstats.com/fight-details/ccee020be2e8...
8212,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,W/L,Open Weight Bout,Submission,1,0:58,No Time Limit,John McCarthy,Guillotine Choke Standing,http://ufcstats.com/fight-details/4b9ae533ccb3...


In [176]:
import pandas as pd
#preprocessing to DF
def add_event_info(df_result: pd.DataFrame, df_events: pd.DataFrame,
                   event_col: str = 'EVENT',
                   date_col: str = 'DATE',
                   location_col: str = 'LOCATION') -> pd.DataFrame:
    """
    מוסיפה ל-df_result את עמודות התאריך והמיקום מתוך df_events לפי התאמת event_col.

    פרמטרים:
    - df_result: טבלה ראשית עם עמודת event_col.
    - df_events: טבלת האירועים עם עמודות event_col, date_col, location_col.
    - event_col: שם העמודה שמשמשת למיזוג (ברירת מחדל 'EVENT').
    - date_col: שם עמודת התאריך ב-df_events (ברירת מחדל 'DATE').
    - location_col: שם עמודת המיקום ב-df_events (ברירת מחדל 'LOCATION').

    מחזירה:
    - עותק של df_result עם עמודות date_col ו-location_col מחוברות.
    """

    df_result = df_result.copy()
    df_events = df_events.copy()

    # ניקוי טקסט בעמודת event_col בשתי הטבלאות
    df_result[event_col] = df_result[event_col].astype(str).str.strip()
    df_events[event_col] = df_events[event_col].astype(str).str.strip()

    # המרת תאריך
    df_events[date_col] = pd.to_datetime(df_events[date_col], errors='coerce')

    # מיזוג טבלאות
    df_merged = df_result.merge(
        df_events[[event_col, date_col, location_col]],
        on=event_col,
        how='left'
    )

    # בדיקה של אירועים שלא נמצאו
    missing_events = df_merged[df_merged[date_col].isna()][event_col].unique()

    return df_merged
#df_result = add_event_info(df_result, df_event)
#df_stat_fighter = add_event_info(df_stat_fighter, df_event)


In [177]:
import pandas as pd
#split name of fight like 'Amit VS Guy' to fighter_=amit,fighter_2=Guy
def split_fighters(df, bout_column='BOUT'):




    # ודא שהערכים הם מחרוזות
    df[bout_column] = df[bout_column].astype(str)

    # פיצול לפי 'vs.' עם רווחים משני הצדדים
    fighters = df[bout_column].str.split(r'\s+vs\.\s+', expand=True)

    # יצירת העמודות החדשות
    df['Fighter_1'] = fighters[0]
    df['Fighter_2'] = fighters[1]

    return df
#df_result = split_fighters(df_result)

##preprosesing to main CSV.





In [178]:
import pandas as pd
import re

def height_to_inches(height_str):
    if pd.isna(height_str):
        return None
    match = re.match(r"(\d+)' ?(\d+)?\"?", height_str)
    if match:
        feet = int(match.group(1))
        inches = int(match.group(2)) if match.group(2) else 0
        total_inches = feet * 12 + inches
        return total_inches
    else:
        try:
            return float(height_str)
        except:
            return None

def add_diff_features_to_main(df_main, df_tott,
                              main_fighter1_col='Fighter_1',
                              main_fighter2_col='Fighter_2',
                              tots_name_col='FIGHTER',
                              tots_height_col='HEIGHT',
                              tots_dob_col='DOB'):
    df_main = df_main.copy()
    df_tott = df_tott.copy()

    # ניקוי שמות
    df_main[main_fighter1_col] = df_main[main_fighter1_col].str.strip().str.lower()
    df_main[main_fighter2_col] = df_main[main_fighter2_col].str.strip().str.lower()
    df_tott[tots_name_col] = df_tott[tots_name_col].str.strip().str.lower()

    # המרת גובה למספר
    df_tott[tots_height_col] = df_tott[tots_height_col].apply(height_to_inches)

    # יצירת מילונים
    height_dict = df_tott.set_index(tots_name_col)[tots_height_col].to_dict()
    dob_dict = df_tott.set_index(tots_name_col)[tots_dob_col].to_dict()

    # מיפוי תכונות
    df_main['Fighter_1_HEIGHT'] = df_main[main_fighter1_col].map(height_dict)
    df_main['Fighter_2_HEIGHT'] = df_main[main_fighter2_col].map(height_dict)
    df_main['Fighter_1_DOB'] = df_main[main_fighter1_col].map(dob_dict)
    df_main['Fighter_2_DOB'] = df_main[main_fighter2_col].map(dob_dict)

    # המרת תאריכים ל-datetime
    df_main['Fighter_1_DOB'] = pd.to_datetime(df_main['Fighter_1_DOB'], errors='coerce')
    df_main['Fighter_2_DOB'] = pd.to_datetime(df_main['Fighter_2_DOB'], errors='coerce')

    # חישוב הפרשים (כאן בלי abs, אבל אפשר להוסיף אם רוצים)
    df_main['diff_HEIGHT'] = df_main['Fighter_1_HEIGHT'] - df_main['Fighter_2_HEIGHT']
    df_main['diff_DOB_days'] = (df_main['Fighter_1_DOB'] - df_main['Fighter_2_DOB']).dt.days

    # אפשר למחוק עמודות ביניים אם לא צריך אותן
    #df_main.drop(columns=['Fighter_1_HEIGHT', 'Fighter_2_HEIGHT', 'Fighter_1_DOB', 'Fighter_2_DOB'], inplace=True)

    return df_main



In [179]:
import numpy as np

def add_date_features_to_df(df, date_col='DATE'):
    df = df.copy()  # יוצרים עותק, לא משנים את המקורי

    # המרת העמודה לפורמט datetime אם עדיין לא
    df[date_col] = pd.to_datetime(df[date_col])

    # חילוץ פיצ'רים בסיסיים
    df['year'] = df[date_col].dt.year
    df['month'] = df[date_col].dt.month
    df['day'] = df[date_col].dt.day
    df['dayofyear'] = df[date_col].dt.dayofyear

    # הוספת פיצ'רים סינוס וקוסינוס ליום בשנה (מחזור שנתי)
    df['sin_dayofyear'] = np.sin(2 * np.pi * df['dayofyear'] / 365)
    df['cos_dayofyear'] = np.cos(2 * np.pi * df['dayofyear'] / 365)

    # פיצ'רים סינוס וקוסינוס לחודש (מחזור שנתי)
    df['sin_month'] = np.sin(2 * np.pi * df['month'] / 12)
    df['cos_month'] = np.cos(2 * np.pi * df['month'] / 12)

    # פיצ'רים סינוס וקוסינוס ליום בחודש (מחזור חודשי)
    df['sin_day'] = np.sin(2 * np.pi * df['day'] / 31)
    df['cos_day'] = np.cos(2 * np.pi * df['day'] / 31)

    return df


In [180]:
def add_main_fight_feature(df):
    df = df.copy()  # יוצרים עותק חדש, לא משנים את המקורי

    def get_last_name(full_name):
        return str(full_name).split()[-1].lower() if pd.notnull(full_name) else ''

    df['Fighter_1_lastname'] = df['Fighter_1'].apply(get_last_name)

    df['Main_Fight'] = df.apply(
        lambda row: int(row['Fighter_1_lastname'] in str(row['EVENT']).lower()),
        axis=1
    )
    df = df.drop(columns=['Fighter_1_lastname'])

    return df


In [181]:
import pandas as pd
import numpy as np
import re

def aggregate_fighter_stats_preserve_order(df_stats):
    df = df_stats.copy()

    # המרת ROUND למספר float
    df['Round_num'] = df['ROUND'].str.extract(r'(\d+)').astype(float)

    # עמודות 'X of Y' לפיצול
    cols_to_split = ['HEAD', 'BODY', 'LEG', 'DISTANCE', 'CLINCH', 'GROUND', 'SIG.STR.', 'TOTAL STR.', 'TD']

    for col in cols_to_split:
        if col in df.columns:
            split_vals = df[col].str.split(' of ', expand=True)
            df[col + '_landed'] = pd.to_numeric(split_vals[0], errors='coerce').fillna(0).astype(np.uint16)
            df[col + '_total'] = pd.to_numeric(split_vals[1], errors='coerce').fillna(0).astype(np.uint16)

    # המרת אחוזים למספרים בין 0 ל-1
    for pct_col in ['SIG.STR. %', 'TD %']:
        if pct_col in df.columns:
            df[pct_col] = (
                df[pct_col].str.replace('%', '', regex=False)
                .replace('---', np.nan)
                .astype(float)
                .fillna(0) / 100
            ).astype(np.float32)

    # עמודות מספריות אחרות
    numeric_cols = ['KD', 'SUB.ATT', 'REV.', 'CTRL']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(np.uint16)

    group_cols = ['EVENT', 'BOUT', 'FIGHTER', 'DATE', 'LOCATION']

    # עמודות לסכום
    sum_cols = [c for c in df.columns if c.endswith('_landed') or c.endswith('_total')] + numeric_cols

    # עמודות לממוצע
    avg_cols = [c for c in ['SIG.STR. %', 'TD %'] if c in df.columns]

    # סכום + ממוצע + ספירת סבבים (Round_num)
    agg_dict = {col: 'sum' for col in sum_cols}
    agg_dict.update({col: 'mean' for col in avg_cols})
    agg_dict['Round_num'] = 'count'

    # ביצוע הקיבוץ ללא מיון (sort=False שומר על סדר הופעה)
    df_agg = df.groupby(group_cols, sort=False).agg(agg_dict).reset_index()
    df_agg.rename(columns={'Round_num': 'rounds_count'}, inplace=True)

    # חישוב יחס סבב ראשון לסבב אחרון עבור עמודות סכום, בצורה וקטורית
    first_vals = df.groupby(group_cols)[sum_cols].first()
    last_vals = df.groupby(group_cols)[sum_cols].last()
    ratio_df = (last_vals / first_vals).replace([np.inf, -np.inf], np.nan).fillna(1).reset_index()
    ratio_df = ratio_df.rename(columns={c: c + '_first_last_ratio' for c in sum_cols})

    # מיזוג סיכום ויחס לפי קבוצות
    df_final = pd.merge(df_agg, ratio_df, on=group_cols, how='left')
    columns_to_drop = [
    'TD_landed_first_last_ratio',
    'TD_total_first_last_ratio',
    'KD_first_last_ratio',
    'SUB.ATT_first_last_ratio',
    'REV._first_last_ratio',
    'CTRL_first_last_ratio'
]
    df_agg = df_final.drop(columns=columns_to_drop, errors='ignore')

    return df_agg

#agg_result = aggregate_fighter_stats_preserve_order(df_stat_fighter_with_Date)


In [182]:
def add_historical_stats_rowwise_fast(df_stat_fighter):
    import pandas as pd
    import numpy as np

    df = df_stat_fighter.copy()
    df['original_index'] = df.index

    df['DATE'] = pd.to_datetime(df['DATE'])
    df = df.sort_values(['FIGHTER', 'DATE']).reset_index(drop=True)

    numeric_cols = df.select_dtypes(include='number').columns.tolist()
    ignore_cols = ['BOUT_ORDER', 'ROUND', 'WINNER', 'original_index']
    numeric_cols = [col for col in numeric_cols if col not in ignore_cols]

    prev_df = pd.DataFrame(index=df.index)
    last3_avg_df = pd.DataFrame(index=df.index)
    all_avg_df = pd.DataFrame(index=df.index)

    for fighter, group in df.groupby('FIGHTER', sort=False):
        idx = group.index
        data = group[numeric_cols]

        prev_vals = data.shift(1).astype(float)
        prev_df.loc[idx, prev_vals.columns + '_prev'] = prev_vals.values

        last3_avg = data.rolling(window=3, min_periods=1).mean().shift(1)
        last3_avg_df.loc[idx, last3_avg.columns + '_last3_avg'] = last3_avg.values

        all_avg = data.expanding(min_periods=1).mean().shift(1)
        all_avg_df.loc[idx, all_avg.columns + '_all_avg'] = all_avg.values

    result = pd.concat([df, prev_df, last3_avg_df, all_avg_df], axis=1)
    result = result.sort_values('original_index').drop(columns=['original_index']).reset_index(drop=True)

    return result


# הנחה שיש לך DataFrame בשם agg_result
#agg_result1 = add_historical_stats_rowwise_fast(agg_result)

# רשימת עמודות למחיקה
columns_to_drop = [
    'HEAD_landed', 'HEAD_total', 'BODY_landed', 'BODY_total',
    'LEG_landed', 'LEG_total', 'DISTANCE_landed', 'DISTANCE_total',
    'CLINCH_landed', 'CLINCH_total', 'GROUND_landed', 'GROUND_total',
    'SIG.STR._landed', 'SIG.STR._total', 'TOTAL STR._landed', 'TOTAL STR._total',
    'TD_landed', 'TD_total', 'KD', 'SUB.ATT', 'REV.', 'CTRL',
    'SIG.STR. %', 'TD %', 'rounds_count',
    'HEAD_landed_first_last_ratio', 'HEAD_total_first_last_ratio',
    'BODY_landed_first_last_ratio', 'BODY_total_first_last_ratio',
    'LEG_landed_first_last_ratio', 'LEG_total_first_last_ratio',
    'DISTANCE_landed_first_last_ratio', 'DISTANCE_total_first_last_ratio',
    'CLINCH_landed_first_last_ratio', 'CLINCH_total_first_last_ratio',
    'GROUND_landed_first_last_ratio', 'GROUND_total_first_last_ratio',
    'SIG.STR._landed_first_last_ratio', 'SIG.STR._total_first_last_ratio',
    'TOTAL STR._landed_first_last_ratio', 'TOTAL STR._total_first_last_ratio'
]

#agg_result1 = agg_result1.drop(columns=columns_to_drop, errors='ignore')


In [183]:
import pandas as pd
import numpy as np

def merge_fighter_features_with_sign_check(df_main, df_features, feature_cols, fighter1_col='Fighter_1', fighter2_col='Fighter_2', date_col='DATE'):
    df_features['DATE'] = pd.to_datetime(df_features['DATE'])

    # המרת הפיצ'רים לעמודות נומריות
    df_features[feature_cols] = df_features[feature_cols].apply(pd.to_numeric, errors='coerce')

    # יצירת מילון פיצ'רים
    feature_dict = {
        (row['FIGHTER'].lower(), row['DATE']): row[feature_cols].values
        for _, row in df_features.iterrows()
    }

    diffs = []
    for _, row in df_main.iterrows():
        f1 = row[fighter1_col]
        f2 = row[fighter2_col]
        date = pd.to_datetime(row[date_col])

        f1_feat = feature_dict.get((str(f1).lower(), date))
        f2_feat = feature_dict.get((str(f2).lower(), date))

        if (f1_feat is not None) and (f2_feat is not None):
            diff = f1_feat - f2_feat
        else:
            diff = np.full(len(feature_cols), np.nan)

        diffs.append(diff)

    diffs_df = pd.DataFrame(diffs, columns=[f'diff_{c}' for c in feature_cols])
    return pd.concat([df_main.reset_index(drop=True), diffs_df], axis=1)
#feature_cols = [col for col in agg_result1.columns if col not in ['FIGHTER', 'DATE']]
#data_n6 = merge_fighter_features_with_sign_check(data_n5, agg_result1, feature_cols)


In [184]:
def add_winning_streak_diff_column(df_result, df_input):
    import pandas as pd
    from collections import defaultdict
    import bisect

    df_result = compute_winning_streaks(df_result)

    df_result["DATE"] = pd.to_datetime(df_result["DATE"])
    df_input = df_input.copy()
    df_input["DATE"] = pd.to_datetime(df_input["DATE"])

    # שמירת סדר המקורי בעמודה זמנית
    df_input['_original_order'] = range(len(df_input))

    streaks_fighter = defaultdict(list)  # key: fighter, value: list of (date, streak)

    for _, row in df_result.iterrows():
        f1 = row["Fighter_1"]
        f2 = row["Fighter_2"]
        date = row["DATE"]
        streak_f1 = row["Streak_F1"]
        streak_f2 = row["Streak_F2"]

        streaks_fighter[f1].append((date, streak_f1))
        streaks_fighter[f2].append((date, streak_f2))

    def get_streak(fighter, fight_date):
        if fighter not in streaks_fighter:
            return 0
        dates_streaks = streaks_fighter[fighter]
        dates = [ds[0] for ds in dates_streaks]
        pos = bisect.bisect_left(dates, fight_date)
        if pos == 0:
            return 0
        return dates_streaks[pos-1][1]

    # ממיינים לפי תאריך לצורך חישוב
    df_input = df_input.sort_values("DATE").reset_index(drop=True)

    streak_diff_list = []
    for _, row in df_input.iterrows():
        f1 = row["Fighter_1"]
        f2 = row["Fighter_2"]
        date = row["DATE"]
        s1 = get_streak(f1, date)
        s2 = get_streak(f2, date)
        streak_diff_list.append(s1 - s2)

    df_input["Winning_Streak_Diff"] = streak_diff_list

    # מחזירים את הסדר המקורי לפי העמודה הזמנית
    df_input = df_input.sort_values('_original_order').reset_index(drop=True)
    df_input.drop(columns=['_original_order'], inplace=True)

    return df_input



# מוסיפים עמודת ההפרש לטבלה הרצויה (יכולה להיות df_result11 או טבלה חדשה)
#df_final = add_winning_streak_diff_column(df_result11, df_result11)


In [185]:
#df_final.head()


In [186]:
def compute_streak_diff(df_input, df_result, fighter1_col='Fighter_1', fighter2_col='Fighter_2'):
    """
    df_input - טבלה חדשה עם עמודות שמות לוחמים
    df_result - טבלה מלאה עם כל הקרבות, חייב לכלול עמודות fighter1, fighter2, outcome (או winner)
    מחזיר סדרה עם הפרש רצף הנצחונות לפי הסדר של df_input
    """

    # נוודא שהשמות אחידים (lowercase ו-strip)
    df_result = df_result.copy()
    df_result[fighter1_col] = df_result[fighter1_col].str.lower().str.strip()
    df_result[fighter2_col] = df_result[fighter2_col].str.lower().str.strip()

    df_input = df_input.copy()
    df_input[fighter1_col] = df_input[fighter1_col].str.lower().str.strip()
    df_input[fighter2_col] = df_input[fighter2_col].str.lower().str.strip()

    def get_winning_streak(fighter_name):
        # מחזיר רצף נצחונות רצוף עד הקרב האחרון בטבלה המלאה
        fights = df_result[
            (df_result[fighter1_col] == fighter_name) | (df_result[fighter2_col] == fighter_name)
        ].sort_index(ascending=False)

        streak = 0
        for _, fight in fights.iterrows():
            # בדיקה מי ניצח (החלף לפי העמודות שלך)
            if 'Winner' in df_result.columns:
                winner = fight['Winner'].lower().strip()
                if winner == fighter_name:
                    streak += 1
                else:
                    break
            else:
                # אם יש עמודת OUTCOME כמו 'W/L' או 'L/W'
                outcome = fight['OUTCOME']
                if fight[fighter1_col] == fighter_name and outcome == 'W/L':
                    streak += 1
                elif fight[fighter2_col] == fighter_name and outcome == 'L/W':
                    streak += 1
                else:
                    break
        return streak

    streak_diffs = []
    for _, row in df_input.iterrows():
        f1 = row[fighter1_col]
        f2 = row[fighter2_col]

        streak1 = get_winning_streak(f1)
        streak2 = get_winning_streak(f2)
        streak_diffs.append(streak1 - streak2)

    return pd.Series(streak_diffs, name='Winning_Streak_Diff')

# קריאה לפונקציה:
  # הטבלה המקורית המלאה עם כל ההיסטוריה

#data_n7['Winning_Streak_Diff'] = compute_streak_diff(data_n6, df_result)


In [187]:
def add_prev_fight_results_and_diff(df_new, df_full,
                                   fighter1_col='Fighter_1',
                                   fighter2_col='Fighter_2',
                                   date_col='EVENT_DATE',
                                   outcome_col='OUTCOME',
                                   method_col='METHOD'):

    def normalize_name(name):
        if pd.isna(name):
            return ""
        return str(name).strip().lower()

    df_full = df_full.copy()
    df_new = df_new.copy()

    df_full[fighter1_col] = df_full[fighter1_col].apply(normalize_name)
    df_full[fighter2_col] = df_full[fighter2_col].apply(normalize_name)
    df_new[fighter1_col] = df_new[fighter1_col].apply(normalize_name)
    df_new[fighter2_col] = df_new[fighter2_col].apply(normalize_name)

    df_full[date_col] = pd.to_datetime(df_full[date_col])
    df_new[date_col] = pd.to_datetime(df_new[date_col])

    results_1 = []
    kos_1 = []
    results_2 = []
    kos_2 = []

    for idx, row in df_new.iterrows():
        f1 = row[fighter1_col]
        f2 = row[fighter2_col]
        fight_date = row[date_col]

        f1_fights = df_full[
            ((df_full[fighter1_col] == f1) | (df_full[fighter2_col] == f1)) &
            (df_full[date_col] < fight_date)
        ].sort_values(by=date_col, ascending=False)

        f2_fights = df_full[
            ((df_full[fighter1_col] == f2) | (df_full[fighter2_col] == f2)) &
            (df_full[date_col] < fight_date)
        ].sort_values(by=date_col, ascending=False)

        # Debug print

        def get_prev_result_and_ko(fighter, fights):
            if fights.empty:
                return None, None
            prev_fight = fights.iloc[0]
            outcome = prev_fight[outcome_col]
            method = prev_fight[method_col]

            if outcome == 'W/L':
                result = 1 if prev_fight[fighter1_col] == fighter else -1
            elif outcome == 'L/W':
                result = 1 if prev_fight[fighter2_col] == fighter else -1
            else:
                result = None

            ko_win = False
            if result == 1 and isinstance(method, str):
                ko_win = ('KO' in method.upper()) or ('TKO' in method.upper())

            return result, ko_win

        r1, k1 = get_prev_result_and_ko(f1, f1_fights)
        r2, k2 = get_prev_result_and_ko(f2, f2_fights)

        results_1.append(r1)
        kos_1.append(k1)
        results_2.append(r2)
        kos_2.append(k2)

    df_new['Fighter_1_prev_result'] = results_1
    df_new['Fighter_1_prev_KO'] = kos_1
    df_new['Fighter_2_prev_result'] = results_2
    df_new['Fighter_2_prev_KO'] = kos_2

    def safe_subtract(a, b):
        if a is None or b is None:
            return None
        return a - b

    df_new['Prev_Result_Diff'] = [safe_subtract(a, b) for a, b in zip(results_1, results_2)]

    return df_new

# קריאה לדוגמה (בדוק שהעמודה בתאריך נקראת 'DATE' גם ב־df שלך)
#data_n8 = add_prev_fight_results_and_diff(data_n7, df_result_with_Date, date_col='DATE')



In [188]:
def add_stance_diff(df_input, df_tott,
                    fighter1_col='Fighter_1',
                    fighter2_col='Fighter_2',
                    fighter_tott_col='FIGHTER',
                    stance_col='STANCE'):
    # יוצרים עותקים נרמלים כדי לא לשנות המקור
    df_temp = df_input.copy()
    df_temp['fighter_1_norm'] = df_temp[fighter1_col].apply(lambda x: x.lower().strip() if isinstance(x, str) else "")
    df_temp['fighter_2_norm'] = df_temp[fighter2_col].apply(lambda x: x.lower().strip() if isinstance(x, str) else "")
    df_tott_temp = df_tott.copy()
    df_tott_temp['fighter_norm'] = df_tott_temp[fighter_tott_col].apply(lambda x: x.lower().strip() if isinstance(x, str) else "")

    # יצירת מילון מיפוי שם -> סטנס
    stance_map = df_tott_temp.set_index('fighter_norm')[stance_col].str.strip().str.lower()

    stance_diff_list = []
    for _, row in df_temp.iterrows():
        s1 = stance_map.get(row['fighter_1_norm'], 'unknown')
        s2 = stance_map.get(row['fighter_2_norm'], 'unknown')
        stance_diff_list.append(f"{s1} - {s2}")

    df_temp['STANCE_DIFF'] = stance_diff_list

    # מוחזיר את הטבלה עם העמודה החדשה, בלי לשנות המקור
    return df_temp.drop(columns=['fighter_1_norm', 'fighter_2_norm'])
#df_with_stance = add_stance_diff(data_n8, df_tott)
#print(df_with_stance[['Fighter_1', 'Fighter_2', 'STANCE_DIFF']])


In [189]:
import numpy as np

# ממיר "--" ל-NaN קודם (אם לא עשית עדיין)
df_tott.replace("--", np.nan, inplace=True)

# מילוי STANCE בערך השכיח
most_common_stance = df_tott['STANCE'].mode().iloc[0]
df_tott['STANCE'].fillna(most_common_stance, inplace=True)

# מילוי HEIGHT בערך השכיח (נניח שמחרוזות בגובה אחידות)
most_common_height = df_tott['HEIGHT'].mode().iloc[0]
df_tott['HEIGHT'].fillna(most_common_height, inplace=True)

# מילוי DOB בערך השכיח
most_common_dob = df_tott['DOB'].mode().iloc[0]
df_tott['DOB'].fillna(most_common_dob, inplace=True)


/tmp/ipython-input-3903756406.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_tott['STANCE'].fillna(most_common_stance, inplace=True)
/tmp/ipython-input-3903756406.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tru

In [190]:
import pandas as pd

# שיכפול הטבלה
df_result_flipped = df_result.copy()

# שינוי סדר הלוחמים בעמודה BOUT
df_result_flipped["BOUT"] = df_result_flipped["BOUT"].apply(
    lambda bout: " vs. ".join(bout.split(" vs. ")[::-1])
)

# החלפת ערכי העמודה OUTCOME
def flip_outcome(outcome):
    if outcome == "W/L":
        return "L/W"
    elif outcome == "L/W":
        return "W/L"
    elif outcome == "Draw":
        return "Draw"
    else:
        return outcome  # במקרה שיש ערכים אחרים

df_result_flipped["OUTCOME"] = df_result_flipped["OUTCOME"].apply(flip_outcome)

# התוצאה הסופית
df_result_flipped.head()


,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,URL
0,UFC 318: Holloway vs. Poirier 3,Dustin Poirier vs. Max Holloway,L/W,Lightweight Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Jason Herzog,Derek Cleary 47 - 48.Sal D'amato 46 - 49.Eric ...,http://ufcstats.com/fight-details/d7507e2741fd...
1,UFC 318: Holloway vs. Poirier 3,Roman Kopylov vs. Paulo Costa,L/W,Middleweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Herb Dean,Derek Cleary 27 - 30.Eric Colon 27 - 30.Chris ...,http://ufcstats.com/fight-details/23c963be9de3...
2,UFC 318: Holloway vs. Poirier 3,Daniel Rodriguez vs. Kevin Holland,W/L,Welterweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Mike Beltran,Sal D'amato 28 - 29.Junichiro Kamijo 28 - 29.C...,http://ufcstats.com/fight-details/5dde61734080...
3,UFC 318: Holloway vs. Poirier 3,Patricio Freire vs. Dan Ige,W/L,Featherweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Dan Miragliotta,Eric Colon 28 - 29.Sal D'amato 28 - 29.Junichi...,http://ufcstats.com/fight-details/acc961fd5cf5...
4,UFC 318: Holloway vs. Poirier 3,Daniel Zellhuber vs. Michael Johnson,L/W,Lightweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Kerry Hatley,Derek Cleary 28 - 29.Eric Colon 28 - 29.Junich...,http://ufcstats.com/fight-details/a95b03e9b5a9...


# **Flipping the dataframes at the location of the fighters and the target to balance the model**

In [191]:
df_result_flipped1 = add_event_info(df_result_flipped, df_event)
df_stat_fighter1 = add_event_info(df_stat_fighter, df_event)
df_result_flipped2 = split_fighters(df_result_flipped1)
df_result_flipped3 = add_diff_features_to_main(df_result_flipped2, df_tott)
df_stat_fighter2 = aggregate_fighter_stats_preserve_order(df_stat_fighter1)
df_stat_fighter3 = add_historical_stats_rowwise_fast(df_stat_fighter2)

# מחיקת עמודות לא נחוצות
columns_to_drop = [
    'HEAD_landed', 'HEAD_total', 'BODY_landed', 'BODY_total',
    'LEG_landed', 'LEG_total', 'DISTANCE_landed', 'DISTANCE_total',
    'CLINCH_landed', 'CLINCH_total', 'GROUND_landed', 'GROUND_total',
    'SIG.STR._landed', 'SIG.STR._total', 'TOTAL STR._landed', 'TOTAL STR._total',
    'TD_landed', 'TD_total', 'KD', 'SUB.ATT', 'REV.', 'CTRL',
    'SIG.STR. %', 'TD %', 'rounds_count',
    'HEAD_landed_first_last_ratio', 'HEAD_total_first_last_ratio',
    'BODY_landed_first_last_ratio', 'BODY_total_first_last_ratio',
    'LEG_landed_first_last_ratio', 'LEG_total_first_last_ratio',
    'DISTANCE_landed_first_last_ratio', 'DISTANCE_total_first_last_ratio',
    'CLINCH_landed_first_last_ratio', 'CLINCH_total_first_last_ratio',
    'GROUND_landed_first_last_ratio', 'GROUND_total_first_last_ratio',
    'SIG.STR._landed_first_last_ratio', 'SIG.STR._total_first_last_ratio',
    'TOTAL STR._landed_first_last_ratio', 'TOTAL STR._total_first_last_ratio'
]
df_stat_fighter4 = df_stat_fighter3.drop(columns=columns_to_drop, errors='ignore')
feature_cols = [col for col in df_stat_fighter4.columns if col not in ['FIGHTER', 'DATE']]
df_result_flipped5 = merge_fighter_features_with_sign_check(df_result_flipped3, df_stat_fighter4, feature_cols)
#df_result_flipped6 = df_result_flipped5.copy()
#df_result_flipped5 = add_winning_streak_diff_column(df_result_flipped5, df_result_flipped5)

df_result_flipped7 = add_prev_fight_results_and_diff(df_result_flipped5, df_result_flipped5, date_col='DATE')
df_result_flipped8 = add_stance_diff(df_result_flipped7, df_tott)
df_result_flipped8.head()

,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,...,diff_SIG.STR._landed_first_last_ratio_all_avg,diff_SIG.STR._total_first_last_ratio_all_avg,diff_TOTAL STR._landed_first_last_ratio_all_avg,diff_TOTAL STR._total_first_last_ratio_all_avg,Fighter_1_prev_result,Fighter_1_prev_KO,Fighter_2_prev_result,Fighter_2_prev_KO,Prev_Result_Diff,STANCE_DIFF
0,UFC 318: Holloway vs. Poirier 3,Dustin Poirier vs. Max Holloway,L/W,Lightweight Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Jason Herzog,Derek Cleary 47 - 48.Sal D'amato 46 - 49.Eric ...,...,-0.582004,-0.518155,-1.008002,-0.692527,-1.0,False,1.0,True,-2.0,southpaw - orthodox
1,UFC 318: Holloway vs. Poirier 3,Roman Kopylov vs. Paulo Costa,L/W,Middleweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Herb Dean,Derek Cleary 27 - 30.Eric Colon 27 - 30.Chris ...,...,0.180889,-0.059999,0.279649,0.038537,1.0,True,-1.0,False,2.0,southpaw - orthodox
2,UFC 318: Holloway vs. Poirier 3,Daniel Rodriguez vs. Kevin Holland,W/L,Welterweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Mike Beltran,Sal D'amato 28 - 29.Junichiro Kamijo 28 - 29.C...,...,0.589276,0.453617,0.525357,0.430989,1.0,True,1.0,False,0.0,southpaw - orthodox
3,UFC 318: Holloway vs. Poirier 3,Patricio Freire vs. Dan Ige,W/L,Featherweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Dan Miragliotta,Eric Colon 28 - 29.Sal D'amato 28 - 29.Junichi...,...,-0.064979,0.406323,0.550827,0.782546,-1.0,False,1.0,True,-2.0,orthodox - orthodox
4,UFC 318: Holloway vs. Poirier 3,Daniel Zellhuber vs. Michael Johnson,L/W,Lightweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Kerry Hatley,Derek Cleary 28 - 29.Eric Colon 28 - 29.Junich...,...,0.466557,0.576710,0.485908,0.568494,-1.0,False,1.0,True,-2.0,switch - southpaw


# **Pre-processing the data of both tables in parallel and merging them**

In [192]:


# Step 1: הוספת מידע אירוע
df_result1 = add_event_info(df_result, df_event)

df_stat_fighter1 = add_event_info(df_stat_fighter, df_event)

# Step 2: פיצול שמות הלוחמים
df_result4 = split_fighters(df_result1)

# Step 3: הוספת פיצ'רים של הבדלי גובה/גיל וכו'
df_result5 = add_diff_features_to_main(df_result4, df_tott)

# Step 4: הוספת תכונות זמן
#df_result6 = add_date_features_to_df(df_result5, date_col='DATE')

# Step 5: תכונה האם הקרב הוא קרב ראשי
#df_result7 = add_main_fight_feature(df_result5)

# שלב אגירת סטטיסטיקות עם שמירת סדר
df_stat_fighter2 = aggregate_fighter_stats_preserve_order(df_stat_fighter1)

# הוספת סטטיסטיקות היסטוריות לכל לוחם
df_stat_fighter3 = add_historical_stats_rowwise_fast(df_stat_fighter2)

# מחיקת עמודות לא נחוצות
columns_to_drop = [
    'HEAD_landed', 'HEAD_total', 'BODY_landed', 'BODY_total',
    'LEG_landed', 'LEG_total', 'DISTANCE_landed', 'DISTANCE_total',
    'CLINCH_landed', 'CLINCH_total', 'GROUND_landed', 'GROUND_total',
    'SIG.STR._landed', 'SIG.STR._total', 'TOTAL STR._landed', 'TOTAL STR._total',
    'TD_landed', 'TD_total', 'KD', 'SUB.ATT', 'REV.', 'CTRL',
    'SIG.STR. %', 'TD %', 'rounds_count',
    'HEAD_landed_first_last_ratio', 'HEAD_total_first_last_ratio',
    'BODY_landed_first_last_ratio', 'BODY_total_first_last_ratio',
    'LEG_landed_first_last_ratio', 'LEG_total_first_last_ratio',
    'DISTANCE_landed_first_last_ratio', 'DISTANCE_total_first_last_ratio',
    'CLINCH_landed_first_last_ratio', 'CLINCH_total_first_last_ratio',
    'GROUND_landed_first_last_ratio', 'GROUND_total_first_last_ratio',
    'SIG.STR._landed_first_last_ratio', 'SIG.STR._total_first_last_ratio',
    'TOTAL STR._landed_first_last_ratio', 'TOTAL STR._total_first_last_ratio'
]
df_stat_fighter4 = df_stat_fighter3.drop(columns=columns_to_drop, errors='ignore')

# Step 6: מיזוג פיצ'רים סטטיסטיים של לוחמים לפי סימן
feature_cols = [col for col in df_stat_fighter4.columns if col not in ['FIGHTER', 'DATE']]
df_result11 = merge_fighter_features_with_sign_check(df_result5, df_stat_fighter4, feature_cols)

# Step 7: פיצ'ר של Winning Streak

#df_result12 = df_result11.copy()
#df_result12 = add_winning_streak_diff_column(df_result11, df_result11)

# Step 8: תוצאת הקרב הקודם והפרש תוצאות
df_result13 = add_prev_fight_results_and_diff(df_result11, df_result11, date_col='DATE')

# Step 9: הפרש סגנונות לחימה (STANCE)
df_result14 = add_stance_diff(df_result13, df_tott)
#df_result_flipped8["red_corner"] = 1
#df_result14["red_corner"] = 0

df_result14 = pd.concat([df_result_flipped8, df_result14], ignore_index=True)
df_result14 = df_result14.sample(frac=1, random_state=42).reset_index(drop=True)



# רשימת העמודות המספריות למילוי לפי ממוצע
numeric_cols = [
    'diff_HEIGHT', 'diff_DOB_days',
    'Fighter_2_HEIGHT', 'Fighter_2_DOB',
    'Fighter_1_HEIGHT', 'Fighter_1_DOB'
]

for col in numeric_cols:
    mean_val = df_result14[col].mean()
    df_result14[col] = df_result14[col].fillna(mean_val)

# רשימת עמודות טקסט למילוי לפי הערך השכיח
text_cols = ['Prev_Result_Diff', 'Fighter_2_prev_result', 'Fighter_2_prev_KO',
             'Fighter_1_prev_result', 'Fighter_1_prev_KO', 'DETAILS', 'REFEREE']

for col in text_cols:
    mode_val = df_result14[col].mode()
    if not mode_val.empty:
        df_result14[col] = df_result14[col].fillna(mode_val[0])
    else:
        # במקרה שאין ערך שכיח (עמודה ריקה לחלוטין) אפשר למלא ב-Unknown
        df_result14[col] = df_result14[col].fillna('Unknown')
import pandas as pd
import numpy as np

# רשימת עמודות עם 2121 ערכי NaN לטיפול
columns_to_fill = [
    col for col in df_result14.columns
    if df_result14[col].isna().sum() == 2121
]

for col in columns_to_fill:
    col_data = df_result14[col]

    if pd.api.types.is_numeric_dtype(col_data):
        unique_vals = col_data.dropna().unique()

        if set(unique_vals).issubset({0, 1}):  # אם בינארי
            mode_val = col_data.mode()
            fill_val = mode_val[0] if not mode_val.empty else 0
            fill_type = "mode (binary)"
        else:  # עמודה מספרית רגילה
            fill_val = col_data.mean()
            fill_type = "mean"

        df_result14[col] = col_data.fillna(fill_val)


    elif pd.api.types.is_string_dtype(col_data) or col_data.dtype == "object":
        mode_val = col_data.mode()
        fill_val = mode_val[0] if not mode_val.empty else "Unknown"
        df_result14[col] = col_data.fillna(fill_val)


df_result14 = df_result14.drop(columns=['EVENT', 'BOUT', 'URL', 'DATE','METHOD','TIME','DETAILS','ROUND','diff_EVENT','diff_BOUT','diff_LOCATION'], errors='ignore')

mapping_WEIGHTCLASS = {
    # Simple direct mappings
    'Lightweight Bout': 'Lightweight Bout',
    'Middleweight Bout': 'Middleweight Bout',
    'Welterweight Bout': 'Welterweight Bout',
    'Featherweight Bout': 'Featherweight Bout',
    'Bantamweight Bout': 'Bantamweight Bout',
    'Light Heavyweight Bout': 'Light Heavyweight Bout',
    'Heavyweight Bout': 'Heavyweight Bout',
    "Women's Flyweight Bout": "Women's Flyweight Bout",
    "Women's Strawweight Bout": "Women's Strawweight Bout",
    'Flyweight Bout': 'Flyweight Bout',
    'Catch Weight Bout': 'Catch Weight Bout',
    "Women's Bantamweight Bout": "Women's Bantamweight Bout",
    'Open Weight Bout': 'Open Weight Bout',

    # UFC Title Bouts mapped to base categories
    'UFC Lightweight Title Bout': 'Lightweight Bout',
    'UFC Flyweight Title Bout': 'Flyweight Bout',
    'UFC Bantamweight Title Bout': 'Bantamweight Bout',
    "UFC Women's Bantamweight Title Bout": "Women's Bantamweight Bout",
    'UFC Welterweight Title Bout': 'Welterweight Bout',
    "UFC Women's Flyweight Title Bout": "Women's Flyweight Bout",
    'UFC Featherweight Title Bout': 'Featherweight Bout',
    'UFC Light Heavyweight Title Bout': 'Light Heavyweight Bout',
    'UFC Middleweight Title Bout': 'Middleweight Bout',
    "UFC Women's Strawweight Title Bout": "Women's Strawweight Bout",
    'UFC Heavyweight Title Bout': 'Heavyweight Bout',
    'UFC Interim Heavyweight Title Bout': 'Heavyweight Bout',
    'UFC Interim Featherweight Title Bout': 'Featherweight Bout',
    'UFC Interim Flyweight Title Bout': 'Flyweight Bout',
    'UFC Interim Bantamweight Title Bout': 'Bantamweight Bout',
    "UFC Women's Featherweight Title Bout": "Women's Featherweight Bout",
    'UFC Interim Lightweight Title Bout': 'Lightweight Bout',
    'UFC Interim Middleweight Title Bout': 'Middleweight Bout',
    'UFC Interim Welterweight Title Bout': 'Welterweight Bout',
    'UFC Interim Light Heavyweight Title Bout': 'Light Heavyweight Bout',

    # Ultimate Fighter mappings (map to base category)
    'Ultimate Fighter 28 Heavyweight Tournament Title Bout': 'Heavyweight Bout',
    "Ultimate Fighter 28 Women's Featherweight Tournament Title Bout": "Women's Featherweight Bout",
    'Ultimate Fighter 27 Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter 27 Featherweight Tournament Title Bout': 'Featherweight Bout',
    'Ultimate Fighter 25 Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter Latin America 3 Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter 23 Light Heavyweight Tournament Title Bout': 'Light Heavyweight Bout',
    "Ultimate Fighter 23 Women's Strawweight Tournament Title Bout": "Women's Strawweight Bout",
    'Ultimate Fighter 22 Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter Latin America 2 Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter Latin America 2 Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter Brazil 4 Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter Brazil 4 Bantamweight Tournament Title Bout': 'Bantamweight Bout',
    'Ultimate Fighter 21 Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter Latin America Featherweight Tournament Title Bout': 'Featherweight Bout',
    'Ultimate Fighter Latin America Bantamweight Tournament Title Bout': 'Bantamweight Bout',
    'Ultimate Fighter China Featherweight Tournament Title Bout': 'Featherweight Bout',
    'Ultimate Fighter 19 Light Heavyweight Tournament Title Bout': 'Light Heavyweight Bout',
    'Ultimate Fighter 19 Middleweight Tournament Title Bout': 'Middleweight Bout',
    'Ultimate Fighter Brazil 3 Heavyweight Tournament Title Bout': 'Heavyweight Bout',
    'Ultimate Fighter Brazil 3 Middleweight Tournament Title Bout': 'Middleweight Bout',
    'Ultimate Fighter China Welterweight Tournament Title Bout': 'Welterweight Bout',
    "Ultimate Fighter 18 Women's Bantamweight Tournament Title Bout": "Women's Bantamweight Bout",
    'Ultimate Fighter 18 Bantamweight Tournament Title Bout': 'Bantamweight Bout',
    'Ultimate Fighter Brazil 2 Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter 17 Middleweight Tournament Title Bout': 'Middleweight Bout',
    'Ultimate Fighter 16 Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter Australia vs. UK Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter Australia vs. UK Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter Brazil 1 Middleweight Tournament Title Bout': 'Middleweight Bout',
    'Ultimate Fighter Brazil 1 Featherweight Tournament Title Bout': 'Featherweight Bout',
    'Ultimate Fighter 15 Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter 14 Featherweight Tournament Title Bout': 'Featherweight Bout',
    'Ultimate Fighter 14 Bantamweight Tournament Title Bout': 'Bantamweight Bout',
    'Ultimate Fighter 13 Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter 12 Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter 11 Middleweight Tournament Title Bout': 'Middleweight Bout',
    'Ultimate Fighter 10 Heavyweight Tournament Title Bout': 'Heavyweight Bout',
    'Ultimate Fighter 9 Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter 9 Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter 8 Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter 8 Light Heavyweight Tournament Title Bout': 'Light Heavyweight Bout',
    'Ultimate Fighter 7 Middleweight Tournament Title Bout': 'Middleweight Bout',
    'Ultimate Fighter 6 Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter 5 Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter 4 Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter 4 Middleweight Tournament Title Bout': 'Middleweight Bout',
    'Ultimate Fighter 3 Light Heavyweight Tournament Title Bout': 'Light Heavyweight Bout',
    'Ultimate Fighter 3 Middleweight Tournament Title Bout': 'Middleweight Bout',
    'Ultimate Fighter 2 Heavyweight Tournament Title Bout': 'Heavyweight Bout',
    'Ultimate Fighter 2 Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter 1 Light Heavyweight Tournament Title Bout': 'Light Heavyweight Bout',
    'Ultimate Fighter 1 Middleweight Tournament Title Bout': 'Middleweight Bout',

    # Other explicit mappings
    'Super Heavyweight Bout': 'Heavyweight Bout',
    'Ultimate Japan 2 Heavyweight Tournament Title Bout': 'Heavyweight Bout',
    'Ultimate Japan Heavyweight Tournament Title Bout': 'Heavyweight Bout',
    'Open Weight Bout': 'Open Weight Bout',
    'UFC 17 Middleweight Tournament Title Bout': 'Middleweight Bout',
    'UFC 15 Heavyweight Tournament Title Bout': 'Heavyweight Bout',
    'UFC 14 Heavyweight Tournament Title Bout': 'Heavyweight Bout',
    'UFC 14 Middleweight Tournament Title Bout': 'Middleweight Bout',
    'UFC 13 Heavyweight Tournament Title Bout': 'Heavyweight Bout',
    'UFC 13 Lightweight Tournament Title Bout': 'Lightweight Bout',
    "Ultimate Ultimate '96 Tournament Title Bout": 'other',
    'UFC 10 Tournament Title Bout': 'other',
    'UFC Superfight Championship Bout': 'other',
    'UFC 8 Tournament Title Bout': 'other',
    "Ultimate Ultimate '95 Tournament Title Bout": 'other',
    'UFC 7 Tournament Title Bout': 'other',
    'UFC 6 Tournament Title Bout': 'other',
    'UFC 5 Tournament Title Bout': 'other',
    'UFC 4 Tournament Title Bout': 'other',
    'UFC 3 Tournament Title Bout': 'other',
    'UFC 2 Tournament Title Bout': 'other',
}

df_result14['WEIGHTCLASS'] = df_result14['WEIGHTCLASS'].map(mapping_WEIGHTCLASS).fillna('other')
# עמודות תאריכים של הלוחמים
df_result14['Fighter_1_AGE'] = (pd.to_datetime('today') - df_result14['Fighter_1_DOB']).dt.days
df_result14['Fighter_2_AGE'] = (pd.to_datetime('today') - df_result14['Fighter_2_DOB']).dt.days

# אפשר להוריד את העמודות המקוריות אם לא צריך אותן
df_result14.drop(['Fighter_1_DOB', 'Fighter_2_DOB'], axis=1, inplace=True)
cat_cols = [
    'OUTCOME',
    'WEIGHTCLASS',
    'TIME FORMAT',
    'REFEREE',
    'LOCATION',
    'STANCE_DIFF',
    'Fighter_1',
    'Fighter_2'
]

# הפיכה לקטגוריה
for col in cat_cols:
    df_result14[col] = df_result14[col].astype('category')
data_n9_clean = df_result14[df_result14['OUTCOME'].isin(['W/L', 'L/W'])]
data_n9_clean['OUTCOME'] =  data_n9_clean['OUTCOME'].map({'W/L':0, 'L/W':1})
import pandas as pd

# שלב 1: בודק עמודות עם ערכים חסרים



/tmp/ipython-input-3969853717.py:83: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_result14[col] = df_result14[col].fillna(mode_val[0])
/tmp/ipython-input-3969853717.py:260: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_n9_clean['OUTCOME'] =  data_n9_clean['OUTCOME'].map({'W/L':0, 'L/W':1})


# **Using a CatBoost model with 25 features **

In [193]:
pip install catboost


In [194]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
# רשימת העמודות למחיקה
wanted_diff_cols = [
    'diff_HEAD_landed_prev', 'diff_HEAD_total_prev', 'diff_BODY_landed_prev', 'diff_BODY_total_prev',
    'diff_LEG_landed_prev', 'diff_LEG_total_prev', 'diff_DISTANCE_landed_prev', 'diff_DISTANCE_total_prev',
    'diff_CLINCH_landed_prev', 'diff_CLINCH_total_prev', 'diff_GROUND_landed_prev', 'diff_GROUND_total_prev',
    'diff_SIG.STR._landed_prev', 'diff_SIG.STR._total_prev', 'diff_TOTAL STR._landed_prev',
    'diff_TD_landed_prev', 'diff_TD_total_prev', 'diff_KD_prev', 'diff_SUB.ATT_prev', 'diff_REV._prev',
    'diff_CTRL_prev', 'diff_SIG.STR. %_prev', 'diff_TD %_prev',
     'diff_HEAD_total_first_last_ratio_prev',
    'diff_BODY_landed_first_last_ratio_prev', 'diff_BODY_total_first_last_ratio_prev',
    'diff_LEG_landed_first_last_ratio_prev', 'diff_LEG_total_first_last_ratio_prev',
    'diff_DISTANCE_landed_first_last_ratio_prev', 'diff_DISTANCE_total_first_last_ratio_prev',
    'diff_CLINCH_landed_first_last_ratio_prev', 'diff_CLINCH_total_first_last_ratio_prev',
    'diff_GROUND_landed_first_last_ratio_prev', 'diff_GROUND_total_first_last_ratio_prev',
    'diff_SIG.STR._landed_first_last_ratio_prev', 'diff_SIG.STR._total_first_last_ratio_prev',
    'diff_TOTAL STR._landed_first_last_ratio_prev', 'diff_TOTAL STR._total_first_last_ratio_prev',
    'diff_HEAD_landed_last3_avg', 'diff_HEAD_total_last3_avg', 'diff_BODY_landed_last3_avg',
    'diff_BODY_total_last3_avg', 'diff_LEG_landed_last3_avg', 'diff_LEG_total_last3_avg',
    'diff_DISTANCE_landed_last3_avg', 'diff_DISTANCE_total_last3_avg', 'diff_CLINCH_landed_last3_avg',
    'diff_CLINCH_total_last3_avg', 'diff_GROUND_landed_last3_avg', 'diff_GROUND_total_last3_avg',
    'diff_SIG.STR._landed_last3_avg', 'diff_SIG.STR._total_last3_avg', 'diff_TOTAL STR._landed_last3_avg',
    'diff_TOTAL STR._total_last3_avg', 'diff_TD_landed_last3_avg', 'diff_TD_total_last3_avg',
    'diff_KD_last3_avg', 'diff_REV._last3_avg', 'diff_CTRL_last3_avg',
    'diff_SIG.STR. %_last3_avg', 'diff_TD %_last3_avg',
    'diff_HEAD_landed_first_last_ratio_last3_avg', 'diff_HEAD_total_first_last_ratio_last3_avg',
    'diff_BODY_landed_first_last_ratio_last3_avg', 'diff_BODY_total_first_last_ratio_last3_avg',
    'diff_LEG_landed_first_last_ratio_last3_avg', 'diff_LEG_total_first_last_ratio_last3_avg',
    'diff_DISTANCE_landed_first_last_ratio_last3_avg', 'diff_DISTANCE_total_first_last_ratio_last3_avg',
    'diff_CLINCH_landed_first_last_ratio_last3_avg', 'diff_CLINCH_total_first_last_ratio_last3_avg',
    'diff_GROUND_landed_first_last_ratio_last3_avg', 'diff_GROUND_total_first_last_ratio_last3_avg',
    'diff_SIG.STR._landed_first_last_ratio_last3_avg', 'diff_SIG.STR._total_first_last_ratio_last3_avg',
    'diff_TOTAL STR._landed_first_last_ratio_last3_avg', 'diff_TOTAL STR._total_first_last_ratio_last3_avg',
    'diff_HEAD_landed_all_avg', 'diff_HEAD_total_all_avg', 'diff_BODY_landed_all_avg',
    'diff_BODY_total_all_avg', 'diff_LEG_landed_all_avg', 'diff_LEG_total_all_avg',
    'diff_DISTANCE_landed_all_avg', 'diff_DISTANCE_total_all_avg', 'diff_CLINCH_landed_all_avg',
    'diff_CLINCH_total_all_avg', 'diff_GROUND_total_all_avg',
    'diff_SIG.STR._landed_all_avg', 'diff_SIG.STR._total_all_avg', 'diff_TOTAL STR._landed_all_avg',
    'diff_TOTAL STR._total_all_avg', 'diff_TD_landed_all_avg', 'diff_TD_total_all_avg',
    'diff_KD_all_avg', 'diff_SUB.ATT_all_avg', 'diff_REV._all_avg', 'diff_CTRL_all_avg',
    'diff_SIG.STR. %_all_avg', 'diff_TD %_all_avg', 'diff_rounds_count_all_avg',
    'diff_HEAD_landed_first_last_ratio_all_avg', 'diff_HEAD_total_first_last_ratio_all_avg',
    'diff_BODY_landed_first_last_ratio_all_avg', 'diff_BODY_total_first_last_ratio_all_avg',
    'diff_LEG_landed_first_last_ratio_all_avg', 'diff_LEG_total_first_last_ratio_all_avg',
     'diff_DISTANCE_total_first_last_ratio_all_avg',
    'diff_CLINCH_landed_first_last_ratio_all_avg', 'diff_CLINCH_total_first_last_ratio_all_avg',
     'diff_GROUND_total_first_last_ratio_all_avg',
    'diff_SIG.STR._landed_first_last_ratio_all_avg', 'diff_SIG.STR._total_first_last_ratio_all_avg',
    'diff_TOTAL STR._landed_first_last_ratio_all_avg','OUTCOME','WEIGHTCLASS','TIME FORMAT','Fighter_1_HEIGHT']

# מחיקת העמודות
# עמודות קטגוריות

X = data_n9_clean.drop(columns=wanted_diff_cols)
y = data_n9_clean['OUTCOME']
cat_features = X.select_dtypes(include='category').columns.tolist()
numeric_cols = df_result14.select_dtypes(include=['number']).columns

for col in numeric_cols:
    mean_val = df_result14[col].mean()
    df_result14[col].fillna(mean_val, inplace=True)


# חלוקה לאימון וטסט
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# יצירת Pool
train_pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features)

# הגדרת המודל
model = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.02,
    depth=6,
    loss_function='MultiClass' if y.nunique() > 2 else 'Logloss',
    early_stopping_rounds=100,
    l2_leaf_reg=5,
    random_strength=1.5,
    border_count=128,
    verbose=100,
    random_seed=42
)





# אימון
model.fit(train_pool, eval_set=test_pool, use_best_model=True)

# חיזוי על קבוצת האימון
train_preds = model.predict(train_pool)
train_preds_labels = train_preds.argmax(axis=1) if train_preds.ndim > 1 else train_preds
train_accuracy = accuracy_score(y_train, train_preds_labels)
print(f"Train Accuracy: {train_accuracy:.4f}")

# חיזוי על קבוצת הטסט
test_preds = model.predict(test_pool)
test_preds_labels = test_preds.argmax(axis=1) if test_preds.ndim > 1 else test_preds
test_accuracy = accuracy_score(y_test, test_preds_labels)
print(f"Test Accuracy: {test_accuracy:.4f}")


/tmp/ipython-input-1684201119.py:64: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_result14[col].fillna(mean_val, inplace=True)


0:	learn: 0.6902904	test: 0.6902992	best: 0.6902992 (0)	total: 135ms	remaining: 4m 29s
100:	learn: 0.5915535	test: 0.5833134	best: 0.5833134 (100)	total: 6.24s	remaining: 1m 57s
200:	learn: 0.5701171	test: 0.5598447	best: 0.5598447 (200)	total: 13.3s	remaining: 1m 59s
300:	learn: 0.5568796	test: 0.5481290	best: 0.5481290 (300)	total: 21.6s	remaining: 2m 1s
400:	learn: 0.5462470	test: 0.5400849	best: 0.5400818 (398)	total: 24.7s	remaining: 1m 38s
500:	learn: 0.5346866	test: 0.5324796	best: 0.5324796 (500)	total: 29.1s	remaining: 1m 27s
600:	learn: 0.5229038	test: 0.5265370	best: 0.5265370 (600)	total: 32.2s	remaining: 1m 14s
700:	learn: 0.5128349	test: 0.5207654	best: 0.5207552 (699)	total: 35.2s	remaining: 1m 5s
800:	learn: 0.5029376	test: 0.5164537	best: 0.5164537 (800)	total: 38.2s	remaining: 57.3s
900:	learn: 0.4959119	test: 0.5141710	best: 0.5141710 (900)	total: 42.8s	remaining: 52.2s
1000:	learn: 0.4884337	test: 0.5120954	best: 0.5120954 (1000)	total: 45.8s	remaining: 45.7s
1100:	

In [195]:
X

,REFEREE,LOCATION,Fighter_1,Fighter_2,Fighter_2_HEIGHT,diff_HEIGHT,diff_DOB_days,diff_TOTAL STR._total_prev,diff_rounds_count_prev,diff_HEAD_landed_first_last_ratio_prev,...,diff_GROUND_landed_first_last_ratio_all_avg,diff_TOTAL STR._total_first_last_ratio_all_avg,Fighter_1_prev_result,Fighter_1_prev_KO,Fighter_2_prev_result,Fighter_2_prev_KO,Prev_Result_Diff,STANCE_DIFF,Fighter_1_AGE,Fighter_2_AGE
0,Chris Tognoni,"Las Vegas, Nevada, USA",bryan caraway,pedro munhoz,66.0,2.0,-764.0,-56.0,0.0,2.020202,...,-0.222222,-0.084887,-1.0,False,1.0,False,-2.0,orthodox - orthodox,14976,14212
1,Marc Goddard,"London, England, United Kingdom",thales leites,gegard mousasi,74.0,-1.0,-1425.0,148.0,3.0,3.750000,...,-0.195011,-0.244062,-1.0,False,-1.0,False,0.0,orthodox - orthodox,16039,14614
2,Herb Dean,"Rio de Janeiro, Rio de Janeiro, Brazil",paul craig,johnny walker,78.0,-3.0,-1585.0,115.0,2.0,0.666667,...,-0.097884,0.235197,-1.0,False,1.0,False,-2.0,orthodox - orthodox,13766,12181
4,Ed Collantes,"Sacramento, California, USA",takeya mizugaki,eddie wineland,67.0,0.0,-193.0,-128.0,-2.0,-0.166667,...,-0.088034,-0.045078,-1.0,False,1.0,True,-2.0,orthodox - orthodox,15208,15015
5,Chris Tognoni,"Las Vegas, Nevada, USA",shavkat rakhmonov,neil magny,75.0,-2.0,2638.0,-173.0,-2.0,0.000000,...,-0.129670,-0.529707,1.0,True,1.0,False,0.0,orthodox - orthodox,11244,13882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16422,Steve Mazzagatti,"Las Vegas, Nevada, USA",dave herman,roy nelson,72.0,4.0,3027.0,-25.0,-1.0,-1.769231,...,0.100000,-0.048878,-1.0,False,-1.0,False,0.0,orthodox - orthodox,14916,17943
16423,Rebin Saber,"Stockholm, Sweden",joel alvarez,danilo belluardo,72.0,3.0,-506.0,NaN,NaN,NaN,...,NaN,NaN,-1.0,False,1.0,False,0.0,orthodox - southpaw,11844,11338
16425,Marc Goddard,"Dublin, Leinster, Ireland",mike king,cathal pendred,73.0,2.0,-1490.0,NaN,NaN,NaN,...,NaN,NaN,1.0,False,1.0,False,0.0,orthodox - orthodox,15342,13852
16426,Kerry Hatley,"Las Vegas, Nevada, USA",tj brown,darren elkins,70.0,-1.0,2197.0,-109.0,-1.0,-2.452381,...,-0.707554,-0.515347,-1.0,False,-1.0,False,0.0,switch - orthodox,12859,15056


# **Example of a battle prediction vector**

In [201]:

#data check\predict vector
data = {
    "EVENT": ["UFC 318: Holloway vs. Poirier 3"],
    "BOUT": ["Conor McGregor vs. Islam Makhachev"],
    "OUTCOME": ["W/L"],
    "WEIGHTCLASS": ["Lightweight Bout"],
    "METHOD": ["Decision - Unanimous"],
    "ROUND": [3],
    "TIME": ["5:00"],
    "TIME FORMAT": ["3 Rnd (5-5-5)"],
    "REFEREE": ["Kerry Hatley"],
    "DETAILS": ["Derek Cleary 28 - 29.Eric Colon 28 - 29.Junich..."],
    "URL": ["http://ufcstats.com/fight-details/a95b03e9b5a9"]
}


data = pd.DataFrame(data)
#preprocessing
# Step 1: הוספת מידע אירוע
data1 = add_event_info(data, df_event)

df_result3 = add_event_info(df_stat_fighter, df_event)

# Step 2: פיצול שמות הלוחמים
data2 = split_fighters(data1)

# Step 3: הוספת פיצ'רים של הבדלי גובה/גיל וכו'
data3 = add_diff_features_to_main(data2, df_tott)



# שלב 1: קבלת שמות הלוחמים והקרב
fighter_1 = data2.loc[0, 'Fighter_1']
fighter_2 = data2.loc[0, 'Fighter_2']

bout_name = f"{fighter_1} vs. {fighter_2}"

# שלב 2: לקיחת שורת בסיס
base_row = df_stat_fighter.iloc[0]

# שלב 3: יצירת שתי שורות חדשות — אחת לכל לוחם
row_1 = base_row.copy()
row_1['FIGHTER'] = fighter_1
row_1['BOUT'] = bout_name


row_2 = base_row.copy()
row_2['FIGHTER'] = fighter_2
row_2['BOUT'] = bout_name
#row_2['Fighter_1'] = fighter_1
#row_2['Fighter_2'] = fighter_2

# שלב 4: יצירת DataFrame משתי השורות
df_new_rows = pd.DataFrame([row_1, row_2])

# שלב 5: חיבור עם df_stat_fighter
df_stat_fighter_extended = pd.concat([df_stat_fighter, df_new_rows], ignore_index=True)

# בדיקה: סינון לפי Fighter == Fighter_1
filtered_fighter_1 = df_stat_fighter_extended[(df_stat_fighter_extended['FIGHTER'] == fighter_1)|(df_stat_fighter_extended['FIGHTER'] == fighter_2)]

# הדפסה לבדיקה


filtered_fighter_1 = add_event_info(filtered_fighter_1, df_event)

filtered_fighter_2 = aggregate_fighter_stats_preserve_order(filtered_fighter_1)

# הוספת סטטיסטיקות היסטוריות לכל לוחם
filtered_fighter_3 = add_historical_stats_rowwise_fast(filtered_fighter_2)






feature_cols = [col for col in filtered_fighter_3.columns if col not in ['FIGHTER', 'DATE']]



data4 = merge_fighter_features_with_sign_check(data3, filtered_fighter_3, feature_cols)
#data5 = data4.copy()


data6 = add_prev_fight_results_and_diff(data4, df_result4, date_col='DATE')
# Step 9: הפרש סגנונות לחימה (STANCE)
data7 = add_stance_diff(data6, df_tott)






# רשימת העמודות המספריות למילוי לפי ממוצע
numeric_cols = [
    'diff_HEIGHT', 'diff_DOB_days',
    'Fighter_2_HEIGHT', 'Fighter_2_DOB',
    'Fighter_1_HEIGHT', 'Fighter_1_DOB'
]

for col in numeric_cols:
    mean_val = data7[col].mean()
    data7[col] = data7[col].fillna(mean_val)

# רשימת עמודות טקסט למילוי לפי הערך השכיח
text_cols = ['Prev_Result_Diff', 'Fighter_2_prev_result', 'Fighter_2_prev_KO',
             'Fighter_1_prev_result', 'Fighter_1_prev_KO', 'DETAILS', 'REFEREE']

for col in text_cols:
    mode_val = data7[col].mode()
    if not mode_val.empty:
        data7[col] = data7[col].fillna(mode_val[0])
    else:
        # במקרה שאין ערך שכיח (עמודה ריקה לחלוטין) אפשר למלא ב-Unknown
        data7[col] = data7[col].fillna('Unknown')
import pandas as pd
import numpy as np

# רשימת עמודות עם 2121 ערכי NaN לטיפול
columns_to_fill = [
    col for col in data7.columns
    if data7[col].isna().sum() == 2121
]

for col in columns_to_fill:
    col_data = data7[col]

    if pd.api.types.is_numeric_dtype(col_data):
        unique_vals = col_data.dropna().unique()

        if set(unique_vals).issubset({0, 1}):  # אם בינארי
            mode_val = col_data.mode()
            fill_val = mode_val[0] if not mode_val.empty else 0
            fill_type = "mode (binary)"
        else:  # עמודה מספרית רגילה
            fill_val = col_data.mean()
            fill_type = "mean"

        data7[col] = col_data.fillna(fill_val)


    elif pd.api.types.is_string_dtype(col_data) or col_data.dtype == "object":
        mode_val = col_data.mode()
        fill_val = mode_val[0] if not mode_val.empty else "Unknown"
        data7[col] = col_data.fillna(fill_val)


df_result14 = data7.drop(columns=['EVENT', 'BOUT', 'URL', 'DATE','METHOD','TIME','DETAILS','ROUND','diff_EVENT','diff_BOUT','diff_LOCATION'], errors='ignore')

mapping_WEIGHTCLASS = {
    # Simple direct mappings
    'Lightweight Bout': 'Lightweight Bout',
    'Middleweight Bout': 'Middleweight Bout',
    'Welterweight Bout': 'Welterweight Bout',
    'Featherweight Bout': 'Featherweight Bout',
    'Bantamweight Bout': 'Bantamweight Bout',
    'Light Heavyweight Bout': 'Light Heavyweight Bout',
    'Heavyweight Bout': 'Heavyweight Bout',
    "Women's Flyweight Bout": "Women's Flyweight Bout",
    "Women's Strawweight Bout": "Women's Strawweight Bout",
    'Flyweight Bout': 'Flyweight Bout',
    'Catch Weight Bout': 'Catch Weight Bout',
    "Women's Bantamweight Bout": "Women's Bantamweight Bout",
    'Open Weight Bout': 'Open Weight Bout',

    # UFC Title Bouts mapped to base categories
    'UFC Lightweight Title Bout': 'Lightweight Bout',
    'UFC Flyweight Title Bout': 'Flyweight Bout',
    'UFC Bantamweight Title Bout': 'Bantamweight Bout',
    "UFC Women's Bantamweight Title Bout": "Women's Bantamweight Bout",
    'UFC Welterweight Title Bout': 'Welterweight Bout',
    "UFC Women's Flyweight Title Bout": "Women's Flyweight Bout",
    'UFC Featherweight Title Bout': 'Featherweight Bout',
    'UFC Light Heavyweight Title Bout': 'Light Heavyweight Bout',
    'UFC Middleweight Title Bout': 'Middleweight Bout',
    "UFC Women's Strawweight Title Bout": "Women's Strawweight Bout",
    'UFC Heavyweight Title Bout': 'Heavyweight Bout',
    'UFC Interim Heavyweight Title Bout': 'Heavyweight Bout',
    'UFC Interim Featherweight Title Bout': 'Featherweight Bout',
    'UFC Interim Flyweight Title Bout': 'Flyweight Bout',
    'UFC Interim Bantamweight Title Bout': 'Bantamweight Bout',
    "UFC Women's Featherweight Title Bout": "Women's Featherweight Bout",
    'UFC Interim Lightweight Title Bout': 'Lightweight Bout',
    'UFC Interim Middleweight Title Bout': 'Middleweight Bout',
    'UFC Interim Welterweight Title Bout': 'Welterweight Bout',
    'UFC Interim Light Heavyweight Title Bout': 'Light Heavyweight Bout',

    # Ultimate Fighter mappings (map to base category)
    'Ultimate Fighter 28 Heavyweight Tournament Title Bout': 'Heavyweight Bout',
    "Ultimate Fighter 28 Women's Featherweight Tournament Title Bout": "Women's Featherweight Bout",
    'Ultimate Fighter 27 Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter 27 Featherweight Tournament Title Bout': 'Featherweight Bout',
    'Ultimate Fighter 25 Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter Latin America 3 Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter 23 Light Heavyweight Tournament Title Bout': 'Light Heavyweight Bout',
    "Ultimate Fighter 23 Women's Strawweight Tournament Title Bout": "Women's Strawweight Bout",
    'Ultimate Fighter 22 Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter Latin America 2 Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter Latin America 2 Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter Brazil 4 Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter Brazil 4 Bantamweight Tournament Title Bout': 'Bantamweight Bout',
    'Ultimate Fighter 21 Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter Latin America Featherweight Tournament Title Bout': 'Featherweight Bout',
    'Ultimate Fighter Latin America Bantamweight Tournament Title Bout': 'Bantamweight Bout',
    'Ultimate Fighter China Featherweight Tournament Title Bout': 'Featherweight Bout',
    'Ultimate Fighter 19 Light Heavyweight Tournament Title Bout': 'Light Heavyweight Bout',
    'Ultimate Fighter 19 Middleweight Tournament Title Bout': 'Middleweight Bout',
    'Ultimate Fighter Brazil 3 Heavyweight Tournament Title Bout': 'Heavyweight Bout',
    'Ultimate Fighter Brazil 3 Middleweight Tournament Title Bout': 'Middleweight Bout',
    'Ultimate Fighter China Welterweight Tournament Title Bout': 'Welterweight Bout',
    "Ultimate Fighter 18 Women's Bantamweight Tournament Title Bout": "Women's Bantamweight Bout",
    'Ultimate Fighter 18 Bantamweight Tournament Title Bout': 'Bantamweight Bout',
    'Ultimate Fighter Brazil 2 Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter 17 Middleweight Tournament Title Bout': 'Middleweight Bout',
    'Ultimate Fighter 16 Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter Australia vs. UK Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter Australia vs. UK Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter Brazil 1 Middleweight Tournament Title Bout': 'Middleweight Bout',
    'Ultimate Fighter Brazil 1 Featherweight Tournament Title Bout': 'Featherweight Bout',
    'Ultimate Fighter 15 Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter 14 Featherweight Tournament Title Bout': 'Featherweight Bout',
    'Ultimate Fighter 14 Bantamweight Tournament Title Bout': 'Bantamweight Bout',
    'Ultimate Fighter 13 Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter 12 Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter 11 Middleweight Tournament Title Bout': 'Middleweight Bout',
    'Ultimate Fighter 10 Heavyweight Tournament Title Bout': 'Heavyweight Bout',
    'Ultimate Fighter 9 Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter 9 Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter 8 Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter 8 Light Heavyweight Tournament Title Bout': 'Light Heavyweight Bout',
    'Ultimate Fighter 7 Middleweight Tournament Title Bout': 'Middleweight Bout',
    'Ultimate Fighter 6 Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter 5 Lightweight Tournament Title Bout': 'Lightweight Bout',
    'Ultimate Fighter 4 Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter 4 Middleweight Tournament Title Bout': 'Middleweight Bout',
    'Ultimate Fighter 3 Light Heavyweight Tournament Title Bout': 'Light Heavyweight Bout',
    'Ultimate Fighter 3 Middleweight Tournament Title Bout': 'Middleweight Bout',
    'Ultimate Fighter 2 Heavyweight Tournament Title Bout': 'Heavyweight Bout',
    'Ultimate Fighter 2 Welterweight Tournament Title Bout': 'Welterweight Bout',
    'Ultimate Fighter 1 Light Heavyweight Tournament Title Bout': 'Light Heavyweight Bout',
    'Ultimate Fighter 1 Middleweight Tournament Title Bout': 'Middleweight Bout',

    # Other explicit mappings
    'Super Heavyweight Bout': 'Heavyweight Bout',
    'Ultimate Japan 2 Heavyweight Tournament Title Bout': 'Heavyweight Bout',
    'Ultimate Japan Heavyweight Tournament Title Bout': 'Heavyweight Bout',
    'Open Weight Bout': 'Open Weight Bout',
    'UFC 17 Middleweight Tournament Title Bout': 'Middleweight Bout',
    'UFC 15 Heavyweight Tournament Title Bout': 'Heavyweight Bout',
    'UFC 14 Heavyweight Tournament Title Bout': 'Heavyweight Bout',
    'UFC 14 Middleweight Tournament Title Bout': 'Middleweight Bout',
    'UFC 13 Heavyweight Tournament Title Bout': 'Heavyweight Bout',
    'UFC 13 Lightweight Tournament Title Bout': 'Lightweight Bout',
    "Ultimate Ultimate '96 Tournament Title Bout": 'other',
    'UFC 10 Tournament Title Bout': 'other',
    'UFC Superfight Championship Bout': 'other',
    'UFC 8 Tournament Title Bout': 'other',
    "Ultimate Ultimate '95 Tournament Title Bout": 'other',
    'UFC 7 Tournament Title Bout': 'other',
    'UFC 6 Tournament Title Bout': 'other',
    'UFC 5 Tournament Title Bout': 'other',
    'UFC 4 Tournament Title Bout': 'other',
    'UFC 3 Tournament Title Bout': 'other',
    'UFC 2 Tournament Title Bout': 'other',
}

data7['WEIGHTCLASS'] = data7['WEIGHTCLASS'].map(mapping_WEIGHTCLASS).fillna('other')
# עמודות תאריכים של הלוחמים
data7['Fighter_1_AGE'] = (pd.to_datetime('today') - data7['Fighter_1_DOB']).dt.days
data7['Fighter_2_AGE'] = (pd.to_datetime('today') - data7['Fighter_2_DOB']).dt.days



# אפשר להוריד את העמודות המקוריות אם לא צריך אותן
data7.drop(['Fighter_1_DOB', 'Fighter_2_DOB'], axis=1, inplace=True)
cat_cols = [
    'OUTCOME',
    'WEIGHTCLASS',
    'TIME FORMAT',
    'REFEREE',
    'LOCATION',
    'STANCE_DIFF',
    'Fighter_1',
    'Fighter_2'
]

# הפיכה לקטגוריה
for col in cat_cols:
    data7[col] = data7[col].astype('category')

numeric_cols = df_result14.select_dtypes(include=['number']).columns

for col in numeric_cols:
    mean_val = df_result14[col].mean()
    df_result14[col].fillna(mean_val, inplace=True)
#data7.loc[0, 'red_corner'] = 0

# בוחרים רק את העמודות שהמודל רוצה
expected_features = model.feature_names_  # רשימת העמודות שהמודל למד
X2 = data7[expected_features].copy()

# מזהים את העמודות הקטגוריות בתוך X2 בלבד
cat_features = X2.select_dtypes(include=['category']).columns.tolist()

# יוצרים Pool עם הנתונים והעמודות הקטגוריות
pool = Pool(data=X2, cat_features=cat_features)

# עושים חיזוי
y_pred = model.predict(pool)

y_pred

/tmp/ipython-input-3311967784.py:295: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_result14[col].fillna(mean_val, inplace=True)


array([1.])